# Learning about Rate Limiting
Lightspeed limits the rate at which you can get data in two ways:
    1. It limits the number of records returned to 100
    2. It limits the frequency that you can request data using a "leaky-bucket" algorythm
In order to get large amounts of data out, you need to deal with both of these. you can [read Lightspeeds doc here](https://developers.lightspeedhq.com/retail/introduction/ratelimits/). This example was written as I was trying to learn about how it works, and how to deal with it. It is included here in case you need to walk through it to0. However, most of this logic will me moved to the pyLightspeed.retail module so you probably don't need to worry about it. 

## Get set up
We are going to use the pyLightspeed retail module, as well as some others that we will need included. 

In [0]:
import sys
sys.path.append('../')
print(sys.path)
from retail.retail import connection

import logging
import requests
import json
import pandas as pd


# Start logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.debug('Start of program')


## Establish a Connection
Best practice would be to store your keys as environment variables, but I have set the connection object up to take both a file and environment variables

In [0]:
KEY_FILE = "D:\Development\.keys\lightspeed_keys.json"

with open(KEY_FILE) as f:
    keys = json.load(f)

store_data = {
            'account_id': keys["account_id"],
            'save_path': 'D:\\Development\\.keys\\'
            }

credentials = {
            'client_id': keys["client_id"],
            'client_secret': keys["client_secret"]
            }

# Creates the connection to lightspeed, and returns a connection object with useful properties
lsr = connection(store_data, credentials)


## Dealing with Pagination

Lightspeed limits records returned to 100, so we need to page through the data and build an entire list of what we want. In this, we are calling the API to get a first set of 100 records, and then we use the data that is in the *@attributes* to get the total count of items, and then loop through to get all the items. We want to build a list of resources that we can .extend as we go, so we set up all_resources as a list.
For educational purposes, this example ignores the rate limiting and any errors. Keep scrolling down to see examples with more handling.

In [0]:
# Going to try to redo this to be universal, but for now, using the resource variable as the name of the API endpoint
resource = 'Category' #This is the name of the access point

# Start at the beginning offset, and set the limit to Lightspeeds max of 100 records
current_offset = 0
current_limit = 100

# Get the first chunk of data
querystring = {'offset':current_offset, 'limit':current_limit}
lightspeed_api = requests.get(lsr.api_url + resource + '.json', params=querystring, headers=lsr.headers)
all_data = lightspeed_api.json()

# Lightspeed API always returns two top level results - the @attributes which includes count, offset, and limit, and a second block that actually has the thing from the API. 
attributes = all_data['@attributes']
all_resources = []
all_resources = all_data[resource]

# We can get the total number of things that are returned from the [@attributes][count]
total_amount = int(all_data['@attributes']['count']) # We need to use these as integers so we can loop, so convert them
current_offset = int(all_data['@attributes']['offset']) + int(all_data['@attributes']['limit'])
current_limit = int(all_data['@attributes']['limit'])


### Looping
Now that we have the first bit, we can look through it. We use the all_resources list we created above and .extend it to add all the records.

In [0]:
while total_amount > current_offset:
    querystring = {'offset':current_offset, 'limit':current_limit}
    all_data = requests.get(lsr.api_url + resource + '.json', params=querystring, headers=lsr.headers).json()
    all_resources.extend(all_data[resource])
    current_offset = current_offset + current_limit


## Dealing with the Rate Limit
The header of the response contains information we need on the drip rate and bucket level. We can take a look at what is in the header, and we can get some variables ready to use in managing it. In this example, we are repeating everything from the example above, but modifying it so that it works with rate limiting. And again, I wrote this mostly as a way to figure out how it works, so it may look janky

In [16]:
# Going to try to redo this to be universal, but for now, using the resource variable as the name of the API endpoint
resource = 'Category' #This is the name of the access point

# Start at the beginning offset, and set the limit to Lightspeeds max of 100 records
current_offset = 0
current_limit = 100
querystring = {'offset':current_offset, 'limit':current_limit}

# Hold the whole content of the request object in the lightspeed_api variable.
# TODO Really any time we create a request, we need to be managing a bunch of error handling.
lightspeed_api = requests.get(lsr.api_url + resource + '.json', params=querystring, headers=lsr.headers)
logging.debug(f"Request Status Code: {lightspeed_api.status_code}")
# And hold just the json returned in all_data
all_data = lightspeed_api.json()

#Take a look at the header, and get some variables we can use
logging.debug(lightspeed_api.headers)
api_drip_rate = float(lightspeed_api.headers['X-LS-API-Drip-Rate'])
# Since the bucket level comes back as a fraction, we pull it appart to get the pieces we need
api_bucket_level, api_bucket_size = [(float(x)) for x in lightspeed_api.headers['X-LS-API-Bucket-Level'].split('/')]

logging.debug(f"Bucket is at {api_bucket_level} with a size of {api_bucket_size} and a current drip rate of {api_drip_rate}")

# Lightspeed API always returns two top level results in the json - the @attributes which includes count, offset, and limit, 
# and a second block that actually has the thing from the API. 
attributes = all_data['@attributes']
all_resources = []
all_resources = all_data[resource]

# We can get the total number of things that are returned from the [@attributes][count]
total_amount = int(all_data['@attributes']['count']) # We need to use these as integers so we can loop, so convert them
current_offset = int(all_data['@attributes']['offset']) + int(all_data['@attributes']['limit'])
current_limit = int(all_data['@attributes']['limit'])

2020-01-28 12:51:06,705 - DEBUG - Starting new HTTPS connection (1): api.lightspeedapp.com:443
2020-01-28 12:51:07,020 - DEBUG - https://api.lightspeedapp.com:443 "GET /API/Account/190211/Category.json?offset=0&limit=100 HTTP/1.1" 200 3074
2020-01-28 12:51:07,031 - DEBUG - Request Status Code: 200
2020-01-28 12:51:07,032 - DEBUG - {'Date': 'Tue, 28 Jan 2020 17:51:07 GMT', 'Content-Type': 'application/json', 'Content-Length': '3074', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=ddc8230b048e55ea8ca5efecae5c395d61580233867; expires=Thu, 27-Feb-20 17:51:07 GMT; path=/; domain=.lightspeedapp.com; HttpOnly; SameSite=Lax; Secure', 'x-frame-options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-LS-Acct-Id': '190211', 'X-LS-OAuth-Client-Id': '71082', 'X-LS-API-Bucket-Level': '1/60', 'X-LS-Shard-Id': '23', 'X-LS-API-Drip-Rate': '1', 'X-LS-Master-System': 'false', 'X-LS-Master-Account': 'false', 'X-LS-Master-Catalog': 'false', 'Content-Encoding

In [0]:
while total_amount > current_offset:
    querystring = {'offset':current_offset, 'limit':current_limit}
    all_data = requests.get(lsr.api_url + resource + '.json', params=querystring, headers=lsr.headers).json()
    all_resources.extend(all_data[resource])
    current_offset = current_offset + current_limit

### Write it out

We can write out the results to a pandas data frame and then to a csv.

In [0]:
# df = pd.DataFrame(all_resources)
# print(df)
# df.to_csv(lsr.save_path+resource+"_export.csv")